In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00


In [ ]:
! wget https://data.deepai.org/gigaword.zip

--2023-04-30 10:11:32--  https://data.deepai.org/gigaword.zip
Resolving data.deepai.org (data.deepai.org)... 185.93.1.250, 2400:52e0:1a00::871:1
Connecting to data.deepai.org (data.deepai.org)|185.93.1.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297054860 (283M) [application/zip]
Saving to: ‘gigaword.zip’

gigaword.zip        100%[===================>] 283.29M  3.95MB/s    in 74s     

2023-04-30 10:12:46 (3.84 MB/s) - ‘gigaword.zip’ saved [297054860/297054860]



In [ ]:
! unzip gigaword.zip

Archive:  gigaword.zip
  inflating: sumdata/DUC2003/input.txt  
  inflating: sumdata/DUC2003/task1_ref0.txt  
  inflating: sumdata/DUC2003/task1_ref1.txt  
  inflating: sumdata/DUC2003/task1_ref2.txt  
  inflating: sumdata/DUC2003/task1_ref3.txt  
  inflating: sumdata/DUC2004/input.txt  
  inflating: sumdata/DUC2004/task1_ref0.txt  
  inflating: sumdata/DUC2004/task1_ref1.txt  
  inflating: sumdata/DUC2004/task1_ref2.txt  
  inflating: sumdata/DUC2004/task1_ref3.txt  
  inflating: sumdata/Giga/input.txt  
  inflating: sumdata/Giga/task1_ref0.txt  
  inflating: sumdata/train/train.article.txt  
  inflating: sumdata/train/train.title.txt  
  inflating: sumdata/train/valid.article.filter.txt  
  inflating: sumdata/train/valid.title.filter.txt  


In [ ]:
! ls sumdata/train

train.article.txt  valid.article.filter.txt
train.title.txt    valid.title.filter.txt


In [ ]:
src_train_texts = open("sumdata/train/train.article.txt").readlines()
tar_train_texts = open("sumdata/train/train.title.txt").readlines()

In [ ]:
src_valid_texts = open("sumdata/train/valid.article.filter.txt").readlines()
tar_valid_texts = open("sumdata/train/valid.title.filter.txt").readlines()

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertModel

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

SOS_id = tokenizer.convert_tokens_to_ids("[CLS]")
EOS_id = tokenizer.convert_tokens_to_ids("[SEP]")
PAD_id = tokenizer.convert_tokens_to_ids("[PAD]")

In [ ]:
src_train_texts[0]

"australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .\n"

In [ ]:
tar_train_texts[0]

'australian current account deficit narrows sharply\n'

In [ ]:
import re
import random

from tqdm import tqdm

train_src = []
train_tar = []

val_src = []
val_tar = []

for src_text, tar_text in zip(tqdm(src_train_texts), tar_train_texts):
  if random.random() > 0.2:
    continue

  src_text = re.sub("\n", "", src_text)
  tar_text = re.sub("\n", "", tar_text)
  
  src_ids = tokenizer.encode(src_text)
  tar_ids = tokenizer.encode(tar_text)

  if 10 <= len(src_ids) <= 48 and 5 <= len(tar_ids) <= 48:
    if random.random() > 0.02:
      train_src.append(src_ids)
      train_tar.append(tar_ids)
    else:
      val_src.append(src_ids)
      val_tar.append(tar_ids)

100%|██████████| 3803957/3803957 [03:38<00:00, 17374.95it/s]


In [ ]:
len(train_src), len(val_src)

(621065, 12596)

In [ ]:
import torch

class SummarizationDataset(torch.utils.data.Dataset):
    def __init__(self, src, tar):
        super(SummarizationDataset).__init__()
        self.src = src
        self.tar = tar

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        return (self.src[idx], self.tar[idx])

def collate_fn_cnn(PAD_id, samples):
    batch_size = len(samples)
    max_src_len = max(len(sample[0]) for sample in samples)
    max_tar_len = max(len(sample[1]) for sample in samples)

    src_tensor = torch.ones((batch_size, max_src_len), dtype=torch.long) * PAD_id
    tar_tensor = torch.ones((batch_size, max_tar_len), dtype=torch.long) * PAD_id

    for (batch_id, (s, t)) in enumerate(samples):
        src_length = len(s)
        tar_length = len(t)

        src_tensor[batch_id][:src_length] = torch.tensor(s)
        tar_tensor[batch_id][:tar_length] = torch.tensor(t)

    return src_tensor, tar_tensor


In [ ]:
from torch.utils.data import DataLoader
from functools import partial

train_dataset = SummarizationDataset(train_src, train_tar)
val_dataset = SummarizationDataset(val_src, val_tar)

In [ ]:
torch.save(train_dataset, "train_dataset.pt")
torch.save(val_dataset, "val_dataset.pt")

# Seq2Seq

https://arxiv.org/pdf/1704.04368.pdf

In [ ]:
from torch import nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.gru = nn.GRU(
            hidden_size
            , hidden_size
            , bidirectional=True
        )

        self.out_last = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)

        out, h = self.gru(x.transpose(0, 1))
        
        out = out.transpose(0, 1)

        return self.out_last(torch.cat((h[0], h[1]), dim=-1)), out

class Decoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Decoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.query = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1)

        self.gru_cell = nn.GRUCell(hidden_size * 3, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)


    def forward(
        self
        , last
        , hidden
        , keys
        , memory # B, L, H
    ):
        batch_size = last.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        last = self.embedding(last).to(device) # B, H

        query = self.query(torch.cat((hidden, last), dim=-1)).unsqueeze(1) # B, 1, H

        energies = self.v((query + keys).tanh()) # B, L, 1
        weights = energies.softmax(dim=1) # B, L, 1

        attn = (memory * weights).sum(dim=1).squeeze(-1) # (B, H)

        context = torch.cat((last, attn), dim=-1)

        hidden = self.gru_cell(context, hidden) # B, H

        out = self.out(hidden) # B, H

        log_probs = F.log_softmax(out, dim=1) # B, V

        return log_probs, hidden

class PG(nn.Module):
    def __init__(self
        , vocab_size
        , hidden_size
    ):
        super(PG, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = Encoder(vocab_size, hidden_size)

        self.decoder = Decoder(vocab_size, hidden_size)

        self.criterion = nn.NLLLoss(reduction='none')

        self.keys = nn.Linear(hidden_size * 2, hidden_size)

    def forward(
        self
        , src_tensor
        , tar_tensor=None # B, L
    ):
        batch_size = src_tensor.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        h, memory = self.encoder(src_tensor)

        keys = self.keys(memory)

        if self.training:
            loss = 0.0

            log_probs = []

            for iter in range(tar_tensor.size(1)-1):
                last = tar_tensor[:, iter]

                lp, h = self.decoder(last, h, keys, memory) # B, vocab_size

                log_probs.append(lp.unsqueeze(2)) # B, vocab_size, 1

            log_probs = torch.cat(log_probs, dim=-1) # B, vocab_size, L

            loss = self.criterion(log_probs, tar_tensor[:, 1:])[tar_tensor[:, 1:] != PAD_id].mean()

            return loss

        else:
            with torch.no_grad():
                generated = [SOS_id]

                for iter in range(100):
                    last = torch.tensor([generated[-1]]).to(device)

                    lp, h = self.decoder(last, h, keys, memory) # 1, vocab_size

                    gen_id = lp[0].argmax().item()

                    generated.append(gen_id)

                    if gen_id == EOS_id:
                      break
                
                return generated

In [ ]:
def train(model, train_loader, val_loader, epochs):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  lr = 1e-3
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  for epoch in range(1, epochs+1):
    n = 0
    for src, tar in tqdm(train_loader):
        src = src.to(device)
        tar = tar.to(device)

        optimizer.zero_grad()

        loss = model(src, tar)

        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()

        if n % 50 == 0 and n !=0: 
          print("Train loss: ", loss.item())

          with torch.no_grad():
            val_loss_sum = 0.0
            for val_src, val_tar in val_loader:
              val_src = val_src.to(device)
              val_tar = val_tar.to(device)

              val_loss_sum += model(val_src, val_tar)

            print("Val loss: ", val_loss_sum.item() / len(val_loader))

        n += 1
      
  return model

In [ ]:
from tqdm import tqdm

train_loader = DataLoader(
    train_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
)

val_loader = DataLoader(
    val_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
    , drop_last=True
)

vocab_size = tokenizer.vocab_size
hidden_size = 384
epochs = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PG(vocab_size, hidden_size).to(device)

model = train(model, train_loader, val_loader, epochs)

  2%|▏         | 50/2424 [00:25<21:51,  1.81it/s]

Train loss:  6.869752407073975


  2%|▏         | 51/2424 [00:37<2:33:47,  3.89s/it]

Val loss:  6.833644409179687


  4%|▍         | 100/2424 [01:00<18:48,  2.06it/s]

...

 99%|█████████▉| 2400/2424 [28:48<00:12,  1.85it/s]

Train loss:  3.2362136840820312


 99%|█████████▉| 2401/2424 [29:00<01:29,  3.91s/it]

Val loss:  3.4281646728515627


100%|██████████| 2424/2424 [29:11<00:00,  1.38it/s]


In [ ]:
torch.save(model, "model.pt")

In [ ]:
model = torch.load("model.pt").to(device)
model.eval()

PG(
  (encoder): Encoder(
    (embedding): Embedding(30522, 384)
    (gru): GRU(384, 384, bidirectional=True)
    (out_last): Linear(in_features=768, out_features=384, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(30522, 384)
    (query): Linear(in_features=768, out_features=384, bias=True)
    (v): Linear(in_features=384, out_features=1, bias=True)
    (gru_cell): GRUCell(1152, 384)
    (out): Linear(in_features=384, out_features=30522, bias=True)
  )
  (criterion): NLLLoss()
  (keys): Linear(in_features=768, out_features=384, bias=True)
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for src, tar in val_loader:
      gen = model(src[0].unsqueeze(0).to(device))

      src = src[0].tolist()
      tar = tar[0].tolist()

      src_str = tokenizer.decode(src[1:src.index(EOS_id)])
      tar_str = tokenizer.decode(tar[1:tar.index(EOS_id)])
      gen_str = tokenizer.decode(gen[1:gen.index(EOS_id)])

      print(f"[SRC] {src_str}")
      print(f"[TAR] {tar_str}")
      print(f"[OUT] {gen_str}")

      break

[SRC] nine african - american employees of southern california edison co. accused the electric utility of race discrimination in a class - action lawsuit filed in federal court in los angeles.
[TAR] black employees suit accuses edison of bias
[OUT] california employees sues # # # employees of race in los angeles court


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Tar:", references[0])
    print("Pred:", predictions[0])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r.split()] for r in references], [p.split() for p in predictions]))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

references = []
predictions = []
model.eval()


for src, tar in tqdm(val_loader):      
      batch_size = src.size(0)

      for i in range(batch_size):
        gen = model(src[i].unsqueeze(0).to(device))
        
        ref = src[i].tolist()
        target = tar[i].tolist()

        src_str = tokenizer.decode(ref[1:ref.index(EOS_id)])
        tar_str = tokenizer.decode(target[1:target.index(EOS_id)])
        if EOS_id in gen:
          gen_str = tokenizer.decode(gen[1:gen.index(EOS_id)])
        else:
          gen_str = tokenizer.decode(gen[1:])

        #print(gen_str)
        
        references.append(tar_str.lower())
        predictions.append(gen_str.lower())

calc_scores(references, predictions)

100%|██████████| 50/50 [02:04<00:00,  2.50s/it]


Tar: mccain obama turn to missouri
Pred: democrats look to the gop presidential candidates
BLEU:  0.10665420158188844
ROUGE:  {'rouge-1': {'f': 0.3311842587449247, 'p': 0.38167750933962047, 'r': 0.30866243417019656}, 'rouge-2': {'f': 0.14921874237609037, 'p': 0.16955758629391393, 'r': 0.1415467600368662}, 'rouge-l': {'f': 0.2985554722204343, 'p': 0.36461407689532827, 'r': 0.2948386304319299}}


# Задание 1, Pointer Generator

Добавьте вычисление метрик BLEU и ROUGE из прошлого семинара. Добавте указатели в модель выше, как в статье https://arxiv.org/pdf/1704.04368.pdf (кроме attention coverage - его не нужно). Обучите новую модель, сравните результаты.

In [ ]:
from torch import nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.gru = nn.GRU(
            hidden_size
            , hidden_size
            , bidirectional=True
        )

        self.out_last = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)

        out, h = self.gru(x.transpose(0, 1))
        
        out = out.transpose(0, 1)

        out_last = self.out_last(torch.cat((h[0], h[1]), dim=-1))

        return out_last, out

class Decoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Decoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.query = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1)
        self.p_h = nn.Linear(2*hidden_size, 1)
        self.p_s = nn.Linear(hidden_size, 1)
        self.p_x = nn.Linear(hidden_size, 1)

        self.gru_cell = nn.GRUCell(hidden_size * 3, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)


    def forward(
        self
        , last
        , hidden
        , keys
        , memory # B, L, H
        , word_mask
    ):
        batch_size = last.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        idx = last

        last = self.embedding(last).to(device) # B, H x_t

        query = self.query(torch.cat((hidden, last), dim=-1)).unsqueeze(1) # B, 1, H

        energies = self.v((query + keys).tanh()) # B, L, 1
        weights = energies.softmax(dim=1) # B, L, 1

        attn = (memory * weights).sum(dim=1).squeeze(-1) # (B, H)

        context = torch.cat((last, attn), dim=-1)

        hidden = self.gru_cell(context, hidden) # B, H


        p_gen = ((self.p_h(attn)+self.p_x(last)+self.p_s(hidden)).sigmoid())

        out = self.out(hidden) # B, H

        log_probs = F.softmax(out, dim=1) # B, V

        a_sum = (weights.squeeze()*word_mask).sum(dim=1).unsqueeze(1)

        log_probs = p_gen*log_probs
        final_dist = log_probs.scatter_add(1, idx.view(-1, 1), (1-p_gen)*a_sum)

        return torch.log(final_dist), hidden

class PG(nn.Module):
    def __init__(self
        , vocab_size
        , hidden_size
    ):
        super(PG, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = Encoder(vocab_size, hidden_size)

        self.decoder = Decoder(vocab_size, hidden_size)

        self.criterion = nn.NLLLoss(reduction='none')

        self.keys = nn.Linear(hidden_size * 2, hidden_size)

    def forward(
        self
        , src_tensor
        , tar_tensor=None # B, L
    ):
        batch_size = src_tensor.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        h, memory = self.encoder(src_tensor)

        keys = self.keys(memory) #

        if self.training:
            loss = 0.0

            log_probs = []

            for iter in range(tar_tensor.size(1)-1):

                last = tar_tensor[:, iter]
                mask = torch.ones(src_tensor.shape).to(device)
                mask *= last.unsqueeze(1)

                mask = (mask == src_tensor)
                mask *= (src_tensor != PAD_id)
                mask *= (src_tensor != EOS_id)
                
                if iter == 0:
                  mask = torch.zeros(src_tensor.shape).to(device)
                mask = torch.tensor(mask, dtype=torch.float)

                lp, h = self.decoder(last, h, keys, memory, mask) # B, vocab_size

                log_probs.append(lp.unsqueeze(2)) # B, vocab_size, 1

            log_probs = torch.cat(log_probs, dim=-1) # B, vocab_size, L

            loss = self.criterion(log_probs, tar_tensor[:, 1:])[tar_tensor[:, 1:] != PAD_id].mean()

            return loss

        else:
            with torch.no_grad():
                generated = [SOS_id]

                for iter in range(100):
                  
                    last = torch.tensor([generated[-1]]).to(device)
                    mask = torch.ones(src_tensor.shape).to(device)
                    mask *= last.unsqueeze(1)

                    mask = (mask == src_tensor)
                    mask *= (src_tensor != PAD_id)
                    mask *= (src_tensor != EOS_id)

                    if iter == 0:
                        mask = torch.zeros(src_tensor.shape).to(device)
                    mask = torch.tensor(mask, dtype=torch.float)

                    lp, h = self.decoder(last, h, keys, memory, mask) # 1, vocab_size

                    gen_id = lp[0].argmax().item()

                    generated.append(gen_id)

                    if gen_id == EOS_id:
                      break
                
                return generated

In [ ]:
def train(model, train_loader, val_loader, epochs):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  lr = 1e-3
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  for epoch in range(1, epochs+1):
    n = 0
    for src, tar in tqdm(train_loader):
        src = src.to(device)
        tar = tar.to(device)

        optimizer.zero_grad()

        loss = model(src, tar)

        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()

        if n % 50 == 0 and n !=0: 
          print("Train loss: ", loss.item())

          with torch.no_grad():
            val_loss_sum = 0.0
            for val_src, val_tar in val_loader:
              val_src = val_src.to(device)
              val_tar = val_tar.to(device)

              val_loss_sum += model(val_src, val_tar)

            print("Val loss: ", val_loss_sum.item() / len(val_loader))

        n += 1
      
  return model

In [ ]:
from tqdm import tqdm

train_loader = DataLoader(
    train_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
)

val_loader = DataLoader(
    val_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
    , drop_last=True
)

vocab_size = tokenizer.vocab_size
hidden_size = 384
epochs = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PG(vocab_size, hidden_size).to(device)

model = train(model, train_loader, val_loader, epochs)

  0%|          | 0/2427 [00:00<?, ?it/s]<ipython-input-15-5c0286ee0320>:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(mask, dtype=torch.float)
  2%|▏         | 50/2427 [00:30<20:34,  1.93it/s]

Train loss:  6.894338607788086


  2%|▏         | 51/2427 [00:43<2:53:52,  4.39s/it]

Val loss:  6.76822584502551


  4%|▍         | 100/2427 [01:10<23:13,  1.67it/s]

...

 99%|█████████▉| 2400/2427 [32:39<00:16,  1.68it/s]

Train loss:  3.579683542251587


 99%|█████████▉| 2401/2427 [32:52<01:56,  4.48s/it]

Val loss:  3.439330120475925


100%|██████████| 2427/2427 [33:06<00:00,  1.22it/s]


In [ ]:
torch.save(model.state_dict(), 'PG_model.pth')

In [ ]:
from tqdm import tqdm

train_loader = DataLoader(
    train_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
)

val_loader = DataLoader(
    val_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
    , drop_last=True
)

vocab_size = tokenizer.vocab_size
hidden_size = 384

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PG(vocab_size, hidden_size).to(device)
model.load_state_dict(torch.load('PG_model.pth'))
model.eval()

PG(
  (encoder): Encoder(
    (embedding): Embedding(30522, 384)
    (gru): GRU(384, 384, bidirectional=True)
    (out_last): Linear(in_features=768, out_features=384, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(30522, 384)
    (query): Linear(in_features=768, out_features=384, bias=True)
    (v): Linear(in_features=384, out_features=1, bias=True)
    (p_h): Linear(in_features=768, out_features=1, bias=True)
    (p_s): Linear(in_features=384, out_features=1, bias=True)
    (p_x): Linear(in_features=384, out_features=1, bias=True)
    (gru_cell): GRUCell(1152, 384)
    (out): Linear(in_features=384, out_features=30522, bias=True)
  )
  (criterion): NLLLoss()
  (keys): Linear(in_features=768, out_features=384, bias=True)
)

In [ ]:
!pip install --upgrade nltk rouge==0.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Tar:", references[0])
    print("Pred:", predictions[0])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r.split()] for r in references], [p.split() for p in predictions]))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
for src, tar in val_loader:
      gen = model(src[0].unsqueeze(0).to(device))

      src = src[0].tolist()
      tar = tar[0].tolist()

      src_str = tokenizer.decode(src[1:src.index(EOS_id)])
      tar_str = tokenizer.decode(tar[1:tar.index(EOS_id)])
      gen_str = tokenizer.decode(gen[1:gen.index(EOS_id)])

      print(f"[SRC] {src_str}")
      print(f"[TAR] {tar_str}")
      print(f"[OUT] {gen_str}")

      break

<ipython-input-15-5c0286ee0320>:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(mask, dtype=torch.float)


[SRC] legislation to enact president barack obama's top domestic goal, remaking us health care, was expected to clear a new senate procedural hurdle tuesday and stay on track for christmas passage.
[TAR] obama health overhaul on track for christmas passage
[OUT] us health care bill to be new senate


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

references = []
predictions = []
model.eval()


for src, tar in tqdm(val_loader):      
      batch_size = src.size(0)

      for i in range(batch_size):
        gen = model(src[i].unsqueeze(0).to(device))
        
        ref = src[i].tolist()
        target = tar[i].tolist()

        src_str = tokenizer.decode(ref[1:ref.index(EOS_id)])
        tar_str = tokenizer.decode(target[1:target.index(EOS_id)])
        if EOS_id in gen:
          gen_str = tokenizer.decode(gen[1:gen.index(EOS_id)])
        else:
          gen_str = tokenizer.decode(gen[1:])
        
        references.append(tar_str.lower())
        predictions.append(gen_str.lower())

  0%|          | 0/49 [00:00<?, ?it/s]<ipython-input-15-5c0286ee0320>:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(mask, dtype=torch.float)
100%|██████████| 49/49 [03:23<00:00,  4.15s/it]


In [ ]:
predictions.index('....................................................................................................',)

10594

In [ ]:
predictions[10594] = 'nothing.'

In [ ]:
calc_scores(references, predictions)

Tar: nba strikes satellite radio deal
Pred: nba games to be built in next month
BLEU:  0.10794665354483139
ROUGE:  {'rouge-1': {'f': 0.33192455977576973, 'p': 0.3797496816091373, 'r': 0.3106829405193504}, 'rouge-2': {'f': 0.15052118667150294, 'p': 0.16925826624616253, 'r': 0.14370407786762548}, 'rouge-l': {'f': 0.3005322029447938, 'p': 0.36308657483920753, 'r': 0.29711538452429326}}


# Задание 2, BERT

Допишите модель выше (без поинтеров), заменив обучаемый с нуля GRU на предобученный DistilBERT. Статья https://arxiv.org/pdf/1908.08345.pdf. Поэкспериментируйте с гиперпараметрами (lr для берта и тд), сравните результаты.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)

#bert.eval()

pass

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
x = torch.tensor(train_dataset[0][0]).unsqueeze(0).to(device)

print(x.size())

h = bert(x).last_hidden_state

h.size()

torch.Size([1, 31])


torch.Size([1, 31, 768])

In [ ]:
from torch import nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        for p in self.bert.parameters():
          p.requires_grad = False
        self.bert.eval()

        self.out_last = nn.Linear(768, hidden_size)

    def forward(self, x):
        h = bert(x).last_hidden_state
        h = self.out_last(h)

        return h[:,-1,:], h

class Decoder(nn.Module):
    def __init__(
        self
        , vocab_size
        , hidden_size
    ):
        super(Decoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.query = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1)

        self.gru_cell = nn.GRUCell(hidden_size*2, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)


    def forward(
        self
        , last
        , hidden
        , keys
        , memory # B, L, H
    ):
        batch_size = last.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        last = self.embedding(last).to(device) # B, H

        query = self.query(torch.cat((hidden, last), dim=-1)).unsqueeze(1) # B, 1, H

        energies = self.v((query + keys).tanh()) # B, L, 1
        weights = energies.softmax(dim=1) # B, L, 1

        attn = (memory * weights).sum(dim=1).squeeze(-1) # (B, H)

        context = torch.cat((last, attn), dim=-1)

        hidden = self.gru_cell(context, hidden) # B, H

        out = self.out(hidden) # B, H

        log_probs = F.log_softmax(out, dim=1) # B, V

        return log_probs, hidden

class PG(nn.Module):
    def __init__(self
        , vocab_size
        , hidden_size
    ):
        super(PG, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = Encoder(vocab_size, hidden_size)

        self.decoder = Decoder(vocab_size, hidden_size)

        self.criterion = nn.NLLLoss(reduction='none')

        self.keys = nn.Linear(hidden_size, hidden_size)

    def forward(
        self
        , src_tensor
        , tar_tensor=None # B, L
    ):
        batch_size = src_tensor.size(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        h, memory = self.encoder(src_tensor)

        keys = self.keys(memory)

        if self.training:
            loss = 0.0

            log_probs = []

            for iter in range(tar_tensor.size(1)-1):
                last = tar_tensor[:, iter]

                lp, h = self.decoder(last, h, keys, memory) # B, vocab_size

                log_probs.append(lp.unsqueeze(2)) # B, vocab_size, 1

            log_probs = torch.cat(log_probs, dim=-1) # B, vocab_size, L

            loss = self.criterion(log_probs, tar_tensor[:, 1:])[tar_tensor[:, 1:] != PAD_id].mean()

            return loss

        else:
            with torch.no_grad():
                generated = [SOS_id]

                for iter in range(100):
                    last = torch.tensor([generated[-1]]).to(device)

                    lp, h = self.decoder(last, h, keys, memory) # 1, vocab_size

                    gen_id = lp[0].argmax().item()

                    generated.append(gen_id)

                    if gen_id == EOS_id:
                      break
                
                return generated

In [ ]:
def train(model, train_loader, val_loader, epochs):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  lr = 1e-3
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  for epoch in range(1, epochs+1):
    n = 0
    for src, tar in tqdm(train_loader):
        src = src.to(device)
        tar = tar.to(device)

        optimizer.zero_grad()

        loss = model(src, tar)

        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()

        if n % 50 == 0 and n !=0: 
          print("Train loss: ", loss.item())

          with torch.no_grad():
            val_loss_sum = 0.0
            for val_src, val_tar in val_loader:
              val_src = val_src.to(device)
              val_tar = val_tar.to(device)

              val_loss_sum += model(val_src, val_tar)

            print("Val loss: ", val_loss_sum.item() / len(val_loader))

        n += 1
      
  return model

In [ ]:
from tqdm import tqdm

train_loader = DataLoader(
    train_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
)

val_loader = DataLoader(
    val_dataset
    , batch_size=256
    , shuffle=True
    , collate_fn=partial(collate_fn_cnn, PAD_id)
    , drop_last=True
)

vocab_size = tokenizer.vocab_size
hidden_size = 384
epochs = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PG(vocab_size, hidden_size).to(device)

model = train(model, train_loader, val_loader, epochs)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  2%|▏         | 50/2426 [01:04<52:16,  1.32s/it]

Train loss:  6.917790412902832


  2%|▏         | 51/2426 [01:31<6:00:29,  9.11s/it]

Val loss:  6.872062215999681


  4%|▍         | 100/2426 [02:33<48:36,  1.25s/it]

...

 99%|█████████▉| 2400/2426 [1:11:36<00:33,  1.29s/it]

Train loss:  3.384972333908081


 99%|█████████▉| 2401/2426 [1:12:03<03:41,  8.86s/it]

Val loss:  3.4170940165617028


100%|██████████| 2426/2426 [1:12:35<00:00,  1.80s/it]


In [ ]:
torch.save(model.state_dict(), 'PG_bert_model.pth')

In [ ]:
model = PG(vocab_size, hidden_size).to(device)
model.load_state_dict(torch.load('PG_bert_model.pth'))
model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PG(
  (encoder): Encoder(
    (bert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dropout): Dropout(

In [ ]:
!pip install --upgrade nltk rouge==0.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Tar:", references[0])
    print("Pred:", predictions[0])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r.split()] for r in references], [p.split() for p in predictions]))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

references = []
predictions = []
model.eval()


for src, tar in tqdm(val_loader):      
      batch_size = src.size(0)

      for i in range(batch_size):
        gen = model(src[i].unsqueeze(0).to(device))
        
        ref = src[i].tolist()
        target = tar[i].tolist()

        src_str = tokenizer.decode(ref[1:ref.index(EOS_id)])
        tar_str = tokenizer.decode(target[1:target.index(EOS_id)])
        if EOS_id in gen:
          gen_str = tokenizer.decode(gen[1:gen.index(EOS_id)])
        else:
          gen_str = tokenizer.decode(gen[1:])
        
        references.append(tar_str.lower())
        predictions.append(gen_str.lower())

calc_scores(references, predictions)

100%|██████████| 49/49 [03:08<00:00,  3.85s/it]


Tar: wall street stocks rise
Pred: wall street stocks rise as cisco earnings
BLEU:  0.09776969846444114
ROUGE:  {'rouge-1': {'f': 0.31807861487073513, 'p': 0.36380509437947267, 'r': 0.2980097613343758}, 'rouge-2': {'f': 0.13673264280743505, 'p': 0.15463508344068744, 'r': 0.13032205056103677}, 'rouge-l': {'f': 0.2860846625398033, 'p': 0.34595755762449, 'r': 0.28354346760787913}}
